In [31]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
def remove_special_char(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('(http://\S+|\S*[^\w\s]\S*)')
    return re.sub(clean, '', text)

In [36]:
csv_dicts = [{"Description":"<html>ghjkhassd 234235 <p>asdhjkhasd bhjkasdnb</p>","Number":"INC8902348","Task_type":"Incident"},{"Description":"Hey there, How you doin.<#3** kasdnb","Number":"INC8102348","Task_type":"Incident"},{"Description":"Hi Team, <html><P>I am fetching issue while login in.","Number":"INC8302348","Task_type":"Incident"}]

In [37]:
import pandas as pd
input_tickets_df = pd.DataFrame(csv_dicts)

In [9]:
input_tickets_df.columns

Index(['Description', 'Number', 'Task_type'], dtype='object')

In [39]:
for col in input_tickets_df.columns:
    for index,row in input_tickets_df.iterrows():
        row[col] = remove_html_tags(row[col])
        row[col] = remove_special_char(row[col])

In [19]:
from bson import json_util
json_util.dumps(input_tickets_df.columns)

'["Description", "Number", "Task_type"]'

In [40]:
input_tickets_df

,Description,Number,Task_type
0,ghjkhassd 234235 asdhjkhasd bhjkasdnb,INC8902348,Incident
1,Hey How you kasdnb,INC8102348,Incident
2,Hi I am fetching issue while login,INC8302348,Incident


In [42]:
json_str = input_tickets_df.to_json(orient='records')

In [44]:
import json 
json_data = json.loads(json_str)

In [45]:
json_data

[{'Description': 'ghjkhassd 234235 asdhjkhasd bhjkasdnb',
  'Number': 'INC8902348',
  'Task_type': 'Incident'},
 {'Description': 'Hey  How you  kasdnb',
  'Number': 'INC8102348',
  'Task_type': 'Incident'},
 {'Description': 'Hi  I am fetching issue while login ',
  'Number': 'INC8302348',
  'Task_type': 'Incident'}]

# ================================================================

In [1]:
data = [{
            "PredictedFieldName" : "Assignment_group",
            "AlgorithmName" : "SVC"
        },  
        {
            "PredictedFieldName" : "Resolution",
            "AlgorithmName" : "RFClassifier"
        }]

In [2]:
data

[{'PredictedFieldName': 'Assignment_group', 'AlgorithmName': 'SVC'},
 {'PredictedFieldName': 'Resolution', 'AlgorithmName': 'RFClassifier'}]

In [3]:
from pymongo import MongoClient
import json
from bson import json_util
client = MongoClient('127.0.0.1',27017)
db = client['TicketTriaging']

In [4]:
customer_tbl = db.TblCustomer
algo_tbl = db.TblAlgorithm
training_tickets = db.TblIncidentTraining
training_hist = db.TblTraining
rt_tickets = db.TblIncidentRT
predicted_tickets = db.TblPredictedData

In [5]:
predicted_fields = []

In [6]:
data

[{'PredictedFieldName': 'Assignment_group', 'AlgorithmName': 'SVC'},
 {'PredictedFieldName': 'Resolution', 'AlgorithmName': 'RFClassifier'}]

In [ ]:
{'PredictedFieldName': 'Assignment Group', 'InputFields': ['Description', 'Task_type']}
{'PredictedFieldName': 'Resolution', 'InputFields': ['Description']}

In [7]:
old_selections = customer_tbl.find_one({'CustomerID':1},{"PredictedFields":1,"_id":0})

In [8]:
old_selections["PredictedFields"]

[{'PredictedFieldName': 'Assignment_group',
  'InputFields': ['Description', 'Task_type'],
  'AlgorithmName': 'SVC'},
 {'PredictedFieldName': 'Resolution',
  'InputFields': ['Description'],
  'AlgorithmName': 'SVC'}]

In [9]:
updated_selections = []

In [10]:
for o_choice in old_selections["PredictedFields"]:
    for n_choice in data:
        tmpdict = {}
        if(o_choice['PredictedFieldName']==n_choice['PredictedFieldName']):
            tmpdict = o_choice
            tmpdict['AlgorithmName']  = n_choice['AlgorithmName']
        if(tmpdict):
            updated_selections.append(tmpdict)
            break

In [11]:
updated_selections

[{'PredictedFieldName': 'Assignment_group',
  'InputFields': ['Description', 'Task_type'],
  'AlgorithmName': 'SVC'},
 {'PredictedFieldName': 'Resolution',
  'InputFields': ['Description'],
  'AlgorithmName': 'RFClassifier'}]

In [9]:
customer_tbl.update_one({'CustomerID':1}, {"$set": {"PredictedFields":data,"FieldsStatus":'Accepted'}}, upsert=False)

In [20]:
#data = request.get_json()
old_selections = customer_tbl.find_one({'CustomerID':1},{"PredictedFields":1,"_id":0})
updated_selections = []
for o_choice in old_selections["PredictedFields"]:
    for n_choice in data:
        tmpdict = {}
        if(o_choice['PredictedFieldName']==n_choice['PredictedFieldName']):
            tmpdict = o_choice
            tmpdict['AlgorithmName']  = n_choice['AlgorithmName']
        if(tmpdict):
            updated_selections.append(tmpdict)
            break
try:
    customer_tbl.update_one({'CustomerID':1}, {"$set": {"PredictedFields":updated_selections}}, upsert=False)
    resp = 'success'
except Exception as e:
    resp = 'failure'

In [21]:
resp

'success'

In [23]:
last_training_dict = training_hist.find_one({"CustomerID" : 1,"FieldsStatus" : "InProgress", "TrainingID": {"$exists": True}},{'_id':0,"TrainingID":1}, sort=[("TrainingID", -1)])
if(last_training_dict):
    last_training_id = last_training_dict['TrainingID']
else:
    #It should be non-Registered Customer
    print("Last InProgress training info not found for the customer.")
    resp = json_util.dumps("failure")
try:
    customer_tbl.update_one({'CustomerID':1,"TrainingID":last_training_id}, {"$set": {"FieldsStatus":status}}, upsert=False)
    resp = 'success'
except Exception as e:
    resp = 'failure'

Last InProgress training info not found for the customer.


# Delete All discarded FieldSelections records from TblCustomer

In [16]:
o_FieldSelections = [ 
        {
            "FieldsStatus" : "Approved",
            "PredictedFields" : [ 
                {
                    "PredictedFieldName" : "Assignment_group",
                    "InputFields" : [ 
                        "Description", 
                        "Priority"
                    ],
                    "AlgorithmName" : "SVC"
                }, 
                {
                    "PredictedFieldName" : "Task_type",
                    "InputFields" : [ 
                        "Description", 
                        "Priority"
                    ],
                    "AlgorithmName" : "SVC"
                }
            ]
        }, 
        {
            "FieldsStatus" : "Discarded",
            "PredictedFields" : [ 
                {
                    "PredictedFieldName" : "Assignment_group",
                    "InputFields" : [ 
                        "Description"
                    ],
                    "AlgorithmName" : "RFClassifier"
                }, 
                {
                    "PredictedFieldName" : "Task_type",
                    "InputFields" : [ 
                        "Description", 
                        "Priority"
                    ],
                    "AlgorithmName" : "SVC"
                }
            ]
        }
    ]


In [17]:
n_FieldSelections = o_FieldSelections
for record in o_FieldSelections:
    if(record['FieldsStatus']=='Discarded'):
        n_FieldSelections.remove(record)

In [18]:
n_FieldSelections

[{'FieldsStatus': 'Approved',
  'PredictedFields': [{'PredictedFieldName': 'Assignment_group',
    'InputFields': ['Description', 'Priority'],
    'AlgorithmName': 'SVC'},
   {'PredictedFieldName': 'Task_type',
    'InputFields': ['Description', 'Priority'],
    'AlgorithmName': 'SVC'}]}]